<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_02_notebooktest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
import numpy as np
import nltk
import pickle

In [36]:
data = pd.read_pickle("drive/MyDrive/data/data_tokenized_sw_free_v2.pkl")
data.head()

,post,tags
Id,,
65526399,node taking hour employee recordsgroovy xmlbel...,[javascript]
65526401,pythonuser usingibased game game begin create ...,[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,object allocated allocate constructremoved des...,[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [38]:
df = data.head(80000).copy()
df.post = df.post.str.split()
df.head()

,post,tags
Id,,
65526399,"[node, taking, hour, employee, recordsgroovy, ...",[javascript]
65526401,"[pythonuser, usingibased, game, game, begin, c...",[python]
65526407,"[remove, ver, powered, whmcomplete, solution, ...","[php, css]"
65526419,"[object, allocated, allocate, constructremoved...",[c++]
65526447,"[react, route, change, createbrowserhistory, b...","[javascript, reactjs]"


In [ ]:
df.shape

(80000, 2)

In [ ]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

texts = df.post.to_list()
id2word = corpora.Dictionary(texts)
id2word.filter_extremes(keep_n = 10000)
bow_corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = [tfidf[text] for text in bow_corpus]
# appliquer ACP

model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.5836768255950322

In [ ]:
from pprint import pprint

print('\nPerplexity: ', model.log_perplexity(tfidf_corpus))

coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

model_topics = model.show_topics(formatted=False)
pprint(model.print_topics(num_words=30))


Perplexity:  -9.32708830006051

Coherence Score:  0.5836768255950322
[(0,
  '0.022*"react" + 0.019*"const" + 0.016*"component" + 0.008*"app" + '
  '0.007*"page" + 0.007*"log" + 0.006*"route" + 0.006*"usestate" + '
  '0.006*"data" + 0.006*"useeffect" + 0.006*"div" + 0.006*"prop" + 0.006*"vue" '
  '+ 0.005*"type" + 0.005*"props" + 0.005*"import" + 0.005*"error" + '
  '0.004*"event" + 0.004*"export" + 0.004*"await" + 0.004*"async" + 0.004*"js" '
  '+ 0.004*"console" + 0.004*"user" + 0.004*"test" + 0.004*"value" + '
  '0.003*"login" + 0.003*"function" + 0.003*"router" + 0.003*"jest"'),
 (1,
  '0.009*"user" + 0.008*"id" + 0.007*"data" + 0.006*"request" + '
  '0.006*"password" + 0.006*"await" + 0.006*"name" + 0.005*"response" + '
  '0.005*"error" + 0.005*"async" + 0.005*"string" + 0.005*"type" + '
  '0.005*"query" + 0.004*"public" + 0.004*"json" + 0.004*"method" + '
  '0.004*"log" + 0.004*"const" + 0.004*"body" + 0.004*"username" + '
  '0.004*"email" + 0.004*"token" + 0.004*"result" + 0.004

In [ ]:
tags = df.tags

def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), 
                                                                  round(prop_topic,4), 
                                                                  topic_keywords]), 
                                                       ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return sent_topics_df

df_topic_sents_keywords = format_topics_sentences(ldamodel=model, 
                                                  corpus=tfidf_corpus)

# Format
df_dominant_topic = df_topic_sents_keywords.copy()
df_dominant_topic = pd.concat([df_dominant_topic, tags], axis=1)
df_dominant_topic.columns = ['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Original_keywords']

# Show
df_dominant_topic.head(10)

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Original_keywords
0,1.0,0.7676,"user, id, data, request, password, await, name...",NaN
1,13.0,0.7710,"array, value, list, data, print, string, resul...",NaN
2,7.0,0.4644,"studio, app, run, android, visual, error, inst...",NaN
3,3.0,0.7944,"std, foo, type, int, thread, value, const, tem...",NaN
4,0.0,0.7874,"react, const, component, app, page, log, route...",NaN
5,3.0,0.4112,"std, foo, type, int, thread, value, const, tem...",NaN
6,7.0,0.5093,"studio, app, run, android, visual, error, inst...",NaN
7,4.0,0.8419,"intent, notification, permission, android, app...",NaN
8,0.0,0.8310,"react, const, component, app, page, log, route...",NaN
9,8.0,0.6467,"df, column, data, plt, dataframe, value, plot,...",NaN


In [ ]:
l = list(enumerate(model[tfidf_corpus[:10]]))
l

[(0, [(1, 0.7673326), (2, 0.014224518), (3, 0.072282195), (8, 0.014744758)]),
 (1,
  [(0, 0.012053657),
   (1, 0.012053657),
   (2, 0.012053657),
   (3, 0.012053657),
   (4, 0.012053657),
   (5, 0.012053657),
   (6, 0.012053657),
   (7, 0.012053657),
   (8, 0.012053657),
   (9, 0.012053658),
   (10, 0.012053658),
   (11, 0.012053657),
   (12, 0.012053657),
   (13, 0.77098054),
   (14, 0.012053657),
   (15, 0.012053657),
   (16, 0.012053657),
   (17, 0.012053657),
   (18, 0.012053657),
   (19, 0.012053657)]),
 (2, [(0, 0.11491717), (7, 0.46425205), (18, 0.29573646)]),
 (3,
  [(0, 0.010820412),
   (1, 0.010820412),
   (2, 0.010820413),
   (3, 0.7944122),
   (4, 0.010820412),
   (5, 0.010820412),
   (6, 0.010820412),
   (7, 0.010820412),
   (8, 0.010820412),
   (9, 0.010820412),
   (10, 0.010820412),
   (11, 0.010820412),
   (12, 0.010820412),
   (13, 0.010820412),
   (14, 0.010820412),
   (15, 0.010820412),
   (16, 0.010820412),
   (17, 0.010820412),
   (18, 0.010820412),
   (19, 0.01082

In [ ]:
df_dominant_topic[df_dominant_topic.Dominant_Topic == 1]

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Original_keywords
0,1.0,0.7676,"user, id, data, request, password, await, name...",NaN
26,1.0,0.8028,"user, id, data, request, password, await, name...",NaN
33,1.0,0.8222,"user, id, data, request, password, await, name...",NaN
41,1.0,0.6415,"user, id, data, request, password, await, name...",NaN
50,1.0,0.8732,"user, id, data, request, password, await, name...",NaN
...,...,...,...,...
79942,1.0,0.5631,"user, id, data, request, password, await, name...",NaN
79943,1.0,0.3439,"user, id, data, request, password, await, name...",NaN
79975,1.0,0.5392,"user, id, data, request, password, await, name...",NaN
79980,1.0,0.4713,"user, id, data, request, password, await, name...",NaN


In [ ]:
sent_topics_sorted_df = pd.DataFrame()

for i, grp in df_topic_sents_keywords.groupby('Dominant_Topic'):
    grp['Dominant_Topic'] = i
    sent_topics_sorted_df = pd.concat([sent_topics_sorted_df, 
                                       grp.sort_values(['Perc_Contribution'],
                                                       ascending = False).head(1)], 
                                            axis=0)

sent_topics_sorted_df.reset_index(inplace = True)

# # Format
sent_topics_sorted_df.columns = ["Topic_Num_Max_Contrib", "Dominant_Topic", "Topic_Perc_Contrib", "Keywords"]

# Show
sent_topics_sorted_df

,Topic_Num_Max_Contrib,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,4498,0.0,0.9013,"react, const, component, app, page, log, route..."
1,13405,1.0,0.8980,"user, id, data, request, password, await, name..."
2,33477,2.0,0.8833,"org, java, springframework, groupid, artifacti..."
3,1277,3.0,0.8889,"std, foo, type, int, thread, value, const, tem..."
4,63724,4.0,0.8677,"intent, notification, permission, android, app..."
5,8028,5.0,0.8643,"cv, bot, img, image, discord, guild, message, ..."
6,44848,6.0,0.8810,"page, chrome, table, db, www, url, website, so..."
7,9298,7.0,0.9072,"studio, app, run, android, visual, error, inst..."
8,15605,8.0,0.8809,"df, column, data, plt, dataframe, value, plot,..."
9,27062,9.0,0.8845,"train, tf, model, shape, training, batch, kera..."


In [ ]:
df_tmp = df_topic_sents_keywords.copy()
df_tmp['Num_Documents'] = df_tmp.groupby('Dominant_Topic').Dominant_Topic.transform('count')
df_tmp['Perc_Documents'] = df_tmp.Num_Documents / df_tmp.shape[0]
df_tmp.set_index('Dominant_Topic', inplace = True) 
df_tmp = df_tmp[['Num_Documents', 'Perc_Documents']]

df_dominant_topics_prop = sent_topics_sorted_df[["Dominant_Topic","Keywords"]].copy()
df_dominant_topics_prop.set_index('Dominant_Topic', inplace = True)
df_dominant_topics_prop = df_dominant_topics_prop.join(df_tmp)
df_dominant_topics_prop.reset_index(inplace = True)
df_dominant_topics_prop.drop_duplicates(subset = ['Dominant_Topic'], inplace = True)
df_dominant_topics_prop.index = range(df_dominant_topics_prop.shape[0])

df_dominant_topics_prop

,Dominant_Topic,Keywords,Num_Documents,Perc_Documents
0,0.0,"value, column, return, like, string, data, obj...",2835,0.2835
1,1.0,"file, error, version, command, module, get, pr...",2595,0.2595
2,2.0,"android, file, version, install, package, modu...",379,0.0379
3,3.0,"file, work, want, like, need, function, using,...",1530,0.1530
4,4.0,"epoch, train, model, keras, loss, layer, input...",106,0.0106
5,5.0,"text, width, column, html, color, want, value,...",1828,0.1828
6,6.0,"button, return, react, const, state, widget, c...",727,0.0727


In [ ]:
from IPython.display import HTML
css_str = '<style> \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str))

import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(model, tfidf_corpus, id2word)
pyLDAvis.save_html(vis, 'lda_tfidf.html')
display(HTML('lda_tfidf.html'))

ModuleNotFoundError: ignored

In [ ]:
lda_model = model
def predict_unsupervised_tags(text):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    
    corpus_new = id2word.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2word[tag[0]] for tag in potential_tags if id2word[tag[0]] in text]
    
    return relevant_tags

for i in range(20):
    print(f"Liste de tags à prédire : {df.tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df.post.iloc[i])}\n\n")

# ajouter les sorties supervisées

Liste de tags à prédire : ['javascript']

Liste des tags prédits par le modèle non supervisé: ['id', 'data', 'name', 'response']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['print', 'name', 'int']


Liste de tags à prédire : ['php', 'css']

Liste des tags prédits par le modèle non supervisé: ['file', 'version']


Liste de tags à prédire : ['c++']

Liste des tags prédits par le modèle non supervisé: ['std']


Liste de tags à prédire : ['javascript', 'reactjs']

Liste des tags prédits par le modèle non supervisé: ['react', 'const', 'app', 'page', 'log', 'route', 'useeffect', 'prop']


Liste de tags à prédire : ['java']

Liste des tags prédits par le modèle non supervisé: ['name', 'string', 'public', 'method']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['file', 'installed', 'vscode', 'running']


Liste de tags à prédire : ['ios', 'flutter']

Liste des tags prédits par le modèle non supervisé: